# Run Agent
- Implement a workflow to write a daily AI newsletter
- see README.md for details


In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

08:14:05 | NewsletterAgent.newsletter_agent | INFO | Test info message
08:14:05 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
08:14:05 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

do_download=True
process_since=None
# process_since='2025-10-05 18:30:00'

try:
    # set up state
    session_id = 'test_newsletter_20251012203454309410'
    step_name = 'step_04_extract_summaries'
    del session_id, step_name
except Exception as e:
    print(e)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                )
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since,
                                 verbose=False
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

🚀 Creating NewsletterAgent...
session_id is not defined
test_newsletter_20251013081415183040


In [6]:
agent.state.get_status()


{'headlines': {'total': 0},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_01_fetch_urls',
  'workflow_complete': False,
  'workflow_status': 'not_started',
  'workflow_status_message': '',
  'progress_percentage': 0.0,
  'max_edits': 2,
  'concurrency': 16,
  'do_download': True,
  'process_since': None},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [7]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


📝 User prompt: 'Show the workflow status'


08:14:31 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting check_workflow_status
08:14:31 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Completed check_workflow_status


⏱️  Total execution time: 8.41s
📊 Final result:
Current workflow status:

- Progress: 0.0% (0/9 complete)
- Summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs (Gather headlines and URLs from sources)

Step-by-step status:
1. Fetch Urls: not_started
2. Filter Urls: not_started
3. Download Articles: not_started
4. Extract Summaries: not_started
5. Rate Articles: not_started
6. Cluster By Topic: not_started
7. Select Sections: not_started
8. Draft Sections: not_started
9. Finalize Newsletter: not_started

What would you like me to do next?
- “Run all steps” (will execute remaining steps in order)
- “Start next step” (will run Step 1)
- “Run Step X” (e.g., “Run Step 1”)
- “Inspect state” (for detailed internal data)


In [8]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


08:14:39 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting check_workflow_status
08:14:39 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Completed check_workflow_status
08:14:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting Step 1: Gather URLs
08:14:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Cleaning download/sources: 
08:14:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Successfully cleaned download/sources
08:14:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Cleaning download/text: 
08:14:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Successfully cleaned download/text
08:14:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Cleaning download/html: 
08:14:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Successfully cleaned download/html
2025-10-13 08:14:40,270 - fetcher_5800803088 - INFO - [fetcher_in

▶ Starting Step 1: step_01_fetch_urls


2025-10-13 08:14:40,476 - fetcher_5800803088 - INFO - [fetch_rss] RSS fetch successful for HackerNoon: 50 articles
2025-10-13 08:14:40,477 - fetcher_5800803088 - INFO - [fetch_html] Fetching HTML from Techmeme: https://www.techmeme.com/river
2025-10-13 08:14:40,553 - fetcher_5800803088 - INFO - [fetch_rss] RSS fetch successful for Ars Technica: 20 articles
2025-10-13 08:14:40,556 - fetcher_5800803088 - INFO - [fetch_rss] Fetching RSS from The Register: https://www.theregister.com/software/ai_ml/headlines.atom
2025-10-13 08:14:40,654 - fetcher_5800803088 - INFO - [fetch_rss] RSS fetch successful for Hacker News: 30 articles
2025-10-13 08:14:40,682 - fetcher_5800803088 - INFO - [fetch_rss] RSS fetch successful for The Register: 50 articles
2025-10-13 08:14:40,683 - fetcher_5800803088 - INFO - [fetch_html] Fetching HTML from The Verge: https://www.theverge.com/ai-artificial-intelligence
2025-10-13 08:14:40,686 - fetcher_5800803088 - INFO - [fetch_html] Fetching HTML from VentureBeat: http

2025-10-13 08:14:42,405 - fetcher_5800803088 - INFO - Downloading https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNhbGllbmNlIjoibWVudGlvbiIsInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMifV0sImJ1bmRsZXMiOnsiYWxsRmVlZGx5TW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9LHsidHlwZSI6InB1YmxpY2F0aW9uQnVja2V0IiwiaWQiOiJkaXNjb3Zlcnk6YWxsLXRvcGljcyIsInRpZXIiOiJ0aWVyMSJ9XSwiY3VzdG9tTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvY2F0ZWdvcnkvZ2xvYmFsLmFsbCJ9XSwiYm9hcmRNb2RlIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC90YWcvZ2xvYmFsLmFsbCJ9XSwiYW5ub3RhdGVkTW9kZSI6W3sidHlwZSI6InN0cmVhbSIsImlkIjoidXNlci82MmVlYmI5Zi03MTUxLTRmOWEtYThjNy05YTU3YjgyMDUzMDgvdGFnL2dsb2JhbC5hbm5vdGF0ZWQifV19LCJyZWZpbmVNb2RlIjoiY3VzdG9tTW9kZSIsImxhbmd1YWdlcyI6WyJlbiJdfQ
2025-10-13 08:14:42,406 - 

...

2025-10-13 08:15:07,021 - fetcher_5800803088 - DEBUG - Added UTC timezone: 2025-10-13 08:15:07+00:00
2025-10-13 08:15:07,022 - fetcher_5800803088 - DEBUG - Converted to UTC: 2025-10-13 08:15:07+00:00
2025-10-13 08:15:07,022 - fetcher_5800803088 - DEBUG - Formatted last_updated: 2025-10-13T08:15:07Z
2025-10-13 08:15:07,023 - fetcher_5800803088 - INFO - Saving HTML to download/sources/Reddit.html
2025-10-13 08:15:07,043 - fetcher_5800803088 - INFO - [fetch_html] Parsing HTML file: download/sources/Reddit.html
2025-10-13 08:15:07,076 - fetcher_5800803088 - INFO - [fetch_html] Parsed HTML file: download/sources/Reddit.html
2025-10-13 08:15:07,077 - fetcher_5800803088 - INFO - [fetch_html] HTML fetch successful for Reddit: 60 articles
2025-10-13 08:15:07,416 - fetcher_5800803088 - DEBUG - performed human like actions
2025-10-13 08:15:07,463 - fetcher_5800803088 - DEBUG - Found last updated time from document.lastModified: 10/13/2025 08:15:07
2025-10-13 08:15:07,464 - fetcher_5800803088 - DE

,source,url
0,Ars Technica,20
1,Bloomberg,31
2,Business Insider,17
3,FT,48
4,Feedly AI,72
5,Hacker News,30
6,HackerNoon,50
7,New York Times,20
8,NewsAPI,31
9,Reddit,50


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,0
1,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,1
2,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",2
3,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,3
4,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,4
...,...,...,...,...,...,...
668,NewsAPI,Dubai Trackless Tram: The Future of Transport ...,https://www.dubaichronicle.com/2025/10/12/duba...,2025-10-12T11:52:06Z,NaN,668
669,NewsAPI,Stricter visa rules have Colorado employers th...,https://www.denverpost.com/2025/10/12/h1b-visa...,2025-10-12T12:00:38Z,NaN,669
670,NewsAPI,Optimizing For AI-Generated Search Results: A ...,https://marketing-tutor.com/blog/optimizing-fo...,2025-10-12T11:34:32Z,NaN,670
671,NewsAPI,Dy minister seeks stronger collaboration to ad...,https://en.antaranews.com/news/385681/dy-minis...,2025-10-12T08:36:39Z,NaN,671


08:15:16 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Completed Step 1: Gathered 784 articles
08:15:19 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting check_workflow_status
08:15:19 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Completed check_workflow_status


⏱️  Total execution time: 45.57s
📊 Final result:
Step 1 completed.

- Result: Fetched URLs and headlines from 17 RSS sources.
- Articles gathered: 673 (stored in persistent state).
- Workflow progress: 1/9 steps complete (11.1%).
- Next step: Step 2 — Filter URLs to AI-related content.

Would you like me to continue with Step 2 now?


In [9]:
pd.DataFrame(state.headline_data) 


,source,title,url,published,rss_summary,id
0,Ars Technica,OpenAI will stop saving most ChatGPT users’ de...,https://arstechnica.com/tech-policy/2025/10/op...,"Fri, 10 Oct 2025 14:58:29 +0000",Court ends controversial order forcing OpenAI ...,0
1,Ars Technica,“Like putting on glasses for the first time”—h...,https://arstechnica.com/science/2025/10/like-p...,"Fri, 10 Oct 2025 11:00:24 +0000",AI is “comically good” at detecting small eart...,1
2,Ars Technica,AI models can acquire backdoors from surprisin...,https://arstechnica.com/ai/2025/10/ai-models-c...,"Thu, 09 Oct 2025 22:03:21 +0000","Anthropic study suggests ""poison"" training att...",2
3,Ars Technica,Bank of England warns AI stock bubble rivals 2...,https://arstechnica.com/ai/2025/10/bank-of-eng...,"Wed, 08 Oct 2025 21:18:30 +0000",Central bank says market concentration hasn't ...,3
4,Ars Technica,Vandals deface ads for AI necklaces that liste...,https://arstechnica.com/tech-policy/2025/10/va...,"Wed, 08 Oct 2025 16:07:29 +0000",Critics attacked subway ads to defend human fr...,4
...,...,...,...,...,...,...
668,NewsAPI,Dubai Trackless Tram: The Future of Transport ...,https://www.dubaichronicle.com/2025/10/12/duba...,2025-10-12T11:52:06Z,NaN,668
669,NewsAPI,Stricter visa rules have Colorado employers th...,https://www.denverpost.com/2025/10/12/h1b-visa...,2025-10-12T12:00:38Z,NaN,669
670,NewsAPI,Optimizing For AI-Generated Search Results: A ...,https://marketing-tutor.com/blog/optimizing-fo...,2025-10-12T11:34:32Z,NaN,670
671,NewsAPI,Dy minister seeks stronger collaboration to ad...,https://en.antaranews.com/news/385681/dy-minis...,2025-10-12T08:36:39Z,NaN,671


In [10]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


,source,count
10,Techmeme,161
4,Feedly AI,72
6,HackerNoon,50
12,The Register,50
9,Reddit,50
3,FT,48
8,NewsAPI,31
1,Bloomberg,31
5,Hacker News,30
13,The Verge,25


In [11]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:15:21 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting Step 2: Filter URLs
08:15:21 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | 🔍 Filtering 673 headlines...
08:15:21 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | 🔄 Checking for duplicates (all urls without date restrictions)
08:15:21 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | 🔍 Filtering 673 articles for dupes.
08:15:22 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | URL deduplication: 490 duplicates filtered, 183 new URLs remain
08:15:22 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | 🔍 Filtering 673 headlines for AI relevance using LLM...
INFO:llm:Initialized LangfuseClient


▶ Starting Step 2: step_02_filter_urls
checking https://arstechnica.com/tech-policy/2025/10/openai-no-longer-forced-to-save-deleted-chats-but-some-users-still-affected/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/science/2025/10/like-putting-on-glasses-for-the-first-time-how-ai-improves-earthquake-detection/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/ai-models-can-acquire-backdoors-from-surprisingly-few-malicious-documents/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/bank-of-england-warns-ai-stock-bubble-rivals-2000-dotcom-peak/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/tech-policy/2025/10/vandals-deface-ads-for-ai-necklaces-that-listen-to-all-your-conversations/
found url, no cutoff set - treating as duplicate
checking https://arstechnica.com/ai/2025/10/insurers-balk-at-paying-out-huge-settlements-for-claims-ag

INFO:llm:Successfully retrieved prompt 'newsagent/filter_urls' from Langfuse
INFO:llm:Parsed prompt 'newsagent/filter_urls': model=gpt-4.1-mini, system_len=459, user_len=954
08:15:22 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Processing 8 chunks with concurrency 16


concurrency:  16


08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Completed Step 2: 72 AI-related articles, 490 duplicates removed


⏱️  Total execution time: 14.64s
📊 Final result:
✅ Step 2 step_02_filter_urls completed successfully! Removed 490 duplicate URLs, classified 183 new articles, found 72 AI-related.


In [12]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting Step 3: Download Articles
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting concurrent scraping of 72 AI-related articles


▶ Starting Step 3: step_03_download_articles


08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Launching browser for 72 URLs with 16 concurrent workers
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Worker 0 fetching 1 of 72 https://restofworld.org/2025/chinese-talent-ai-race/
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | scrape_url(https://restofworld.org/2025/chinese-talent-ai-race/)
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | scraping https://restofworld.org/2025/chinese-talent-ai-race/ to download/html
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Downloading https://restofworld.org/2025/chinese-talent-ai-race/
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Worker 1 fetching 2 of 72 https://biztoc.com/x/a5a414eea376ff05
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | scrape_url(https://biztoc.com/x/a5a414eea376ff05)
08:15:36 | NewsletterAgent.tes

08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Rate limiting domain hackernoon.com, will retry later (need to wait 2.0s)
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Worker 11 re-queued rate-limited URL: https://hackernoon.com/the-hidden-infrastructure-trap-stalling-enterprise-ai
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Worker 12 fetching 12 of 72 https://www.fool.com/investing/2025/10/13/if-youd-invested-10000-each-in-nvidia-and-palantir/
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | scrape_url(https://www.fool.com/investing/2025/10/13/if-youd-invested-10000-each-in-nvidia-and-palantir/)
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | scraping https://www.fool.com/investing/2025/10/13/if-youd-invested-10000-each-in-nvidia-and-palantir/ to download/html
08:15:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Downloading https://www.f

08:16:10 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Saving HTML to download/html/Salesforce_announces_Agentforce_360_as_enterprise_AI_competition_heats_up.html
08:16:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Worker 8 completed https://techcrunch.com/2025/10/13/salesforce-announces-agentforce-360-as-enterprise-ai-competition-heats-up/ with status: 200
08:16:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Worker 8 fetching 19 of 72 https://hackernoon.com/from-months-to-minutes-i-discovered-just-in-time-app-generation-with-imagine-with-claude
08:16:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | scrape_url(https://hackernoon.com/from-months-to-minutes-i-discovered-just-in-time-app-generation-with-imagine-with-claude)
08:16:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | scraping https://hackernoon.com/from-months-to-minutes-i-discovered-just-in-time-app-generation-with-imagine-with-claud

...

concurrency:  16


08:20:25 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Inserted 4 new site records
08:20:25 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Updated 70 URL records with final URLs


Starting with 70 rows...
Processing 70 files...
Reading and truncating files to 8192 tokens using text-embedding-3-large tokenizer...
Error reading : [Errno 2] No such file or directory: ''
Error reading : [Errno 2] No such file or directory: ''
Getting embeddings for 68 texts...


08:20:28 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Completed Step 3: Downloaded 70 articles


Creating indexed similarity matrix...
Finding pairs with similarity > 0.925...
Filtering dataframe...
  Pair: 43 vs 48
    43: Feedly AI - The AI Co-Founder You'll Never See Join the Board
    48: HackerNoon - The AI Co-Founder You'll Never See Join the Board
  Pair: 44 vs 49
    44: Feedly AI - From Months to Minutes: I Discovered Just-In-Time App Generation with "Imagine with Claude"
    49: HackerNoon - From Months to Minutes: I Discovered Just-In-Time App Generation with "Imagine with Claude"
  Pair: 56 vs 22
    56: Techmeme - Sources: London-based self-driving tech startup Wayve is in early stage talks with Microsoft and SoftBank to raise between $1B and $2B, valuing it at ~$8B
    22: Feedly AI - Sources: London-based self-driving tech startup Wayve is in early stage talks with Microsoft and SoftBank to raise between $1B and $2B, valuing it at ~$8B (Financial Times)
Removing 3 rows due to high similarity 
Final dataframe has 67 rows (removed 3 rows)
⏱️  Total execution time: 292

In [13]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["status"]!=200])




,id,source,title,status,final_url
10,218,HackerNoon,The AI Co-Founder You'll Never See Join the Board,exists,https://hackernoon.com/the-ai-co-founder-youll-never-see-join-the-board?source=rss
13,220,HackerNoon,"From Months to Minutes: I Discovered Just-In-Time App Generation with ""Imagine with Claude""",exists,https://hackernoon.com/from-months-to-minutes-i-discovered-just-in-time-app-generation-with-imagine-with-claude?source=rss
65,140,Feedly AI,We launched an AI agent and it flopped. Here’s what we learned—and why we’re trying again,403,https://www.fastcompany.com/91414267/we-launched-an-ai-agent-and-it-flopped-heres-why-were-trying-again-and-what-we-learned_ai-agent-failure-lessons-learned
66,153,Feedly AI,AI Is Juicing the Economy. Is It Making American Workers More Productive?,skipped,https://www.wsj.com/tech/ai/ai-worker-productivity-economy-77498195


In [14]:
headline_df = state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df[['id', 'source', 'title', 'status', 'final_url']].loc[headline_df["html_path"]==""])



,id,source,title,status,final_url
65,140,Feedly AI,We launched an AI agent and it flopped. Here’s what we learned—and why we’re trying again,403,https://www.fastcompany.com/91414267/we-launched-an-ai-agent-and-it-flopped-heres-why-were-trying-again-and-what-we-learned_ai-agent-failure-lessons-learned
66,153,Feedly AI,AI Is Juicing the Economy. Is It Making American Workers More Productive?,skipped,https://www.wsj.com/tech/ai/ai-worker-productivity-economy-77498195


In [15]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

08:20:28 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Processing 67 AI articles for summarization
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_summaries' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_summaries': model=gpt-4.1-mini, system_len=1273, user_len=43
08:20:29 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Using model 'gpt-4.1-mini' for summarization
08:20:29 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting summarization for 67 articles
08:20:29 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Processing 67 chunks with concurrency 16


▶ Starting Step 4: step_04_extract_summaries
concurrency:  16


08:20:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Extracting metadata from HTML files for 67 articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/item_distiller' from Langfuse
INFO:llm:Parsed prompt 'newsagent/item_distiller': model=gpt-5-mini, system_len=1365, user_len=151
08:20:42 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Processing 67 chunks with concurrency 16


concurrency:  16
⏱️  Total execution time: 65.06s
📊 Final result:
✅ Step 4 step_04_extract_summaries completed successfully! Generated AI-powered summaries for 67/67 articles.
💾 Summaries stored in headline DataFrame.


In [16]:
# if we get a refusal, examine prompt and delete rows
bad_stuff =["What\'s the Best Way ",
            "AI could make it eas",
            "AI can design toxic ",
           ]
print([len(s) for s in bad_stuff])
headline_df = state.headline_df
headline_df.loc[(headline_df["title"].str[:20].isin(bad_stuff))]



[20, 20, 20]


,source,title,url,published,rss_summary,id,isAI,status,final_url,html_path,last_updated,text_path,content_length,domain,site_name,reputation,summary,description,tags,short_summary


In [17]:
# remove bad stuff
state.headline_df_to_dict(headline_df.loc[~headline_df["title"].str[:20].isin(bad_stuff)])


In [18]:
# User prompt to run workflow
# user_prompt = "Run step 5, Rate articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("rate_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:21:34 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Rating 67 AI articles using fn_rate_articles
08:21:34 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Calculating article ratings for 67 articles
08:21:34 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Rating recency
08:21:34 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Rating spam probability
08:21:34 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Initialized LangfuseClient
08:21:34 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
08:21:34 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


▶ Starting Step 5: step_05_rate_articles
concurrency:  16


08:21:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | low quality articles: {0.0: 57, 1.0: 3, 1.0129987330277146e-05: 1, 9.357622968840175e-14: 1, 1.1176086870894152e-14: 1, 0.2227001469810028: 1, 0.9924227025175841: 1}
08:21:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
08:21:36 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1790, user_len=240


concurrency:  16


08:21:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | on topic articles: {1.0: 50, 0.6224593113235651: 2, 0.8519528238199198: 2, 0.008577484930474293: 1, 8.592166619174109e-10: 1, 0.9999038520641101: 1, 0.9986749217974152: 1, 0.9770226167277287: 1, 0.5621765025686553: 1, 0.999512127527129: 1, 0.047425874568977164: 1, 0.0: 1, 0.49999999904767284: 1, 0.014063626520379485: 1}
08:21:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Rating importance probability
08:21:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Successfully retrieved prompt 'newsagent/rate_importance' from Langfuse
08:21:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Parsed prompt 'newsagent/rate_importance': model=gpt-4.1, system_len=2145, user_len=252


...

,id,site_name,title,bradley_terry,bt_z
33,33,SlashGear,Google Wants Its Latest AI Tool To Find And Fix Vulnerable Code Before It Becomes A Problem,19.949079,3.144459
5,5,The Register,UK waves £750M supercomputer contract at HPC builders,9.014928,1.420971
8,8,SiliconANGLE,Salesforce bets big on agents with platform overhaul,8.252994,1.300872
20,20,VentureBeat,Salesforce bets on AI 'agents' to fix what it calls a $7 billion problem in enterprise software,7.666602,1.208442
7,7,Business Insider,"In Big Tech's knotty game of AI Twister, what happens when a giant slips?",7.192913,1.133778
...,...,...,...,...,...
64,64,Biztoc,Condo Adviser: Impact of artificial intelligence on communal living is not yet clear,-10.684869,-1.684194
28,28,The Wall Street Journal,AI Is Juicing the Economy. Is It Making American Workers More Productive?,-11.153174,-1.758011
59,59,Reddit,Another World (1991) game intro remade with A.I.,-11.596871,-1.827948
24,24,Financial Times,The AI trade: do geopolitics matter?,-12.127180,-1.911538


08:28:53 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Top 10 ids: [33, 5, 8, 20, 7, 32, 31, 58, 11, 9]
08:28:53 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | After round 11/13: 
08:28:53 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Number of ranking changes: 31
08:28:53 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Sum of absolute ranking changes: 762.0 (avg rank chg 11.72)
08:28:53 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | last_two: 9.323076923076922, prev_two: 9.338461538461539
08:28:53 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | ---------------------------------------------------
08:28:53 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Running round 12 of max 13
08:28:53 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | ---------------------------------------------------
08:28:53 | NewsletterAgent.test_newsletter_20251013081415183040 | INF

,id,site_name,title,bradley_terry,bt_z
33,33,SlashGear,Google Wants Its Latest AI Tool To Find And Fix Vulnerable Code Before It Becomes A Problem,19.978520,3.131629
5,5,The Register,UK waves £750M supercomputer contract at HPC builders,9.046912,1.418102
8,8,SiliconANGLE,Salesforce bets big on agents with platform overhaul,8.286092,1.298843
20,20,VentureBeat,Salesforce bets on AI 'agents' to fix what it calls a $7 billion problem in enterprise software,7.699180,1.206845
7,7,Business Insider,"In Big Tech's knotty game of AI Twister, what happens when a giant slips?",7.225326,1.132569
...,...,...,...,...,...
64,64,Biztoc,Condo Adviser: Impact of artificial intelligence on communal living is not yet clear,-10.751468,-1.685291
28,28,The Wall Street Journal,AI Is Juicing the Economy. Is It Making American Workers More Productive?,-11.220298,-1.758780
59,59,Reddit,Another World (1991) game intro remade with A.I.,-11.663668,-1.828278
24,24,Financial Times,The AI trade: do geopolitics matter?,-12.193957,-1.911400


08:30:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Top 10 ids: [33, 5, 8, 20, 7, 32, 31, 58, 11, 9]
08:30:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | After round 12/13: 
08:30:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Number of ranking changes: 19
08:30:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Sum of absolute ranking changes: 364.0 (avg rank chg 5.60)
08:30:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | last_two: 8.661538461538461, prev_two: 8.169230769230769
08:30:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Increase in avg rank change, stopping
08:30:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Low rated articles: 2
08:30:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | low rated article: We launched an AI agent and it flopped. Here’s what we learned—and why we’re trying again -1.2847199316646076
08:30:40 | N

⏱️  Total execution time: 548.06s
📊 Final result:
✅ Step 5 step_05_rate_articles completed successfully! Rated 63 articles with average rating 6.5/10.
⭐ High quality articles (≥7.0): 28
💾 Ratings stored in persistent state.


In [19]:
headline_df=state.headline_df
with pd.option_context('display.max_columns', None, 'display.width', None, 'display.max_colwidth', None  ):
    display(headline_df.sort_values("rating", ascending=False)[['site_name', 'title', 'rating', 'short_summary']])


,site_name,title,rating,short_summary
1,Financial Times,Who owns OpenAI? Blockbuster deals complicate investor payouts,11.206399,"Microsoft, OpenAI staff and its nonprofit arm hold nearly 90% of startup equity; a recent Nvidia deal will dilute shareholders and complicate investor payouts."
2,Financial Times,Regulating military use of AI is in everyone’s interest,10.808522,"The article argues blanket bans on military AI are ineffective and urges all states to agree on ethical, legal best practices and international collaboration to manage risks and ensure responsible defense-sector AI use."
5,The Register,UK waves £750M supercomputer contract at HPC builders,10.673698,"The UK government plans to invest up to £750 million in the Next National Supercomputing Service at the University of Edinburgh, budgeting about £300–330 million for the system and targeting 2027 operation and top-five global ranking."
33,SlashGear,Google Wants Its Latest AI Tool To Find And Fix Vulnerable Code Before It Becomes A Problem,10.635428,"Google's DeepMind developed CodeMender, a Gemini-powered agent that autonomously detects and fixes software vulnerabilities—using static and dynamic analysis, fuzzing and SMT solvers—and has patched 72 flaws in large open-source projects; human experts still review high-stakes changes."
0,Financial Times,"Sources: London-based self-driving tech startup Wayve is in early stage talks with Microsoft and SoftBank to raise between $1B and $2B, valuing it at ~$8B",10.559936,"London-based autonomous driving startup Wayve is in early talks with Microsoft and SoftBank to raise $1–2 billion, seeking roughly an $8 billion valuation to accelerate its AI-driven vehicle technology development."
...,...,...,...,...
39,Wired,"Programming in Assembly Is Brutal, Beautiful, and Maybe Even a Path to Better AI",3.220829,"An article argues that programming in assembly—illustrated by Chris Sawyer’s 1999 Rollercoaster Tycoon—can optimize hardware-level performance and potentially advance AI by enabling more efficient, low-level implementations for models such as DeepSeek."
62,Marketing Tutor,Optimizing For AI-Generated Search Results: A Universal Guide | The Marketing Tutor,2.825868,"The Marketing Tutor guide says AI-driven search uses NLP, contextual understanding, intent recognition, machine learning and structured data to improve relevance; businesses should optimize high-quality authoritative content for voice and visual search, mobile-first design, structured data, and ongoing analytics-driven updates."
61,Speckyboy,Why The Grumpy Designer Is Wary of AI’s Confidence,2.621147,"The Grumpy Designer warns that AI tools often present confidently phrased but potentially inaccurate answers without clear citations or safety warnings, risking user misguidance; it recommends transparency, source citations, risk warnings, and multiple solution paths."
60,The Server Side,20 Tough AWS AI Practitioner Certification Exam Questions and Answers,2.112176,"Article offers 20 practice questions to prepare for the AWS Certified AI Practitioner exam, covering neural networks, AWS AI services (SageMaker Clarify, Bedrock, Artifact), transformer architectures, model security, prompt engineering, evaluation metrics, and responsible AI practices."


In [20]:
for row in headline_df.sort_values("rating", ascending=False).itertuples():
    display(Markdown(f"""
{row.rating:.1f} - [{row.title}]({row.url}) - {row.site_name}

{row.short_summary}
    """))
    


11.2 - [Who owns OpenAI? Blockbuster deals complicate investor payouts](https://www.ft.com/content/61ab5bc8-a125-4246-9761-80473028a99e) - Financial Times

Microsoft, OpenAI staff and its nonprofit arm hold nearly 90% of startup equity; a recent Nvidia deal will dilute shareholders and complicate investor payouts.
    


10.8 - [Regulating military use of AI is in everyone’s interest](https://www.ft.com/content/c8dbfb26-1c89-4e28-b728-d2c39725a87d) - Financial Times

The article argues blanket bans on military AI are ineffective and urges all states to agree on ethical, legal best practices and international collaboration to manage risks and ensure responsible defense-sector AI use.
    


10.7 - [UK waves £750M supercomputer contract at HPC builders](https://go.theregister.com/feed/www.theregister.com/2025/10/13/edinburgh_supercomputer_procurement/) - The Register

The UK government plans to invest up to £750 million in the Next National Supercomputing Service at the University of Edinburgh, budgeting about £300–330 million for the system and targeting 2027 operation and top-five global ranking.
    


10.6 - [Google Wants Its Latest AI Tool To Find And Fix Vulnerable Code Before It Becomes A Problem](https://www.slashgear.com/1994542/ai-coding-security-google-deepmind-codemender/) - SlashGear

Google's DeepMind developed CodeMender, a Gemini-powered agent that autonomously detects and fixes software vulnerabilities—using static and dynamic analysis, fuzzing and SMT solvers—and has patched 72 flaws in large open-source projects; human experts still review high-stakes changes.
    


10.6 - [Sources: London-based self-driving tech startup Wayve is in early stage talks with Microsoft and SoftBank to raise between $1B and $2B, valuing it at ~$8B](https://t.co/qNXpYVQUuY) - Financial Times

London-based autonomous driving startup Wayve is in early talks with Microsoft and SoftBank to raise $1–2 billion, seeking roughly an $8 billion valuation to accelerate its AI-driven vehicle technology development.
    

...


3.2 - [Programming in Assembly Is Brutal, Beautiful, and Maybe Even a Path to Better AI](https://www.wired.com/story/programming-assembly-artificial-intelligence/) - Wired

An article argues that programming in assembly—illustrated by Chris Sawyer’s 1999 Rollercoaster Tycoon—can optimize hardware-level performance and potentially advance AI by enabling more efficient, low-level implementations for models such as DeepSeek.
    


2.8 - [Optimizing For AI-Generated Search Results: A Universal Guide | The Marketing Tutor](https://marketing-tutor.com/blog/optimizing-for-ai-generated-search-results-a-universal-guide/) - Marketing Tutor

The Marketing Tutor guide says AI-driven search uses NLP, contextual understanding, intent recognition, machine learning and structured data to improve relevance; businesses should optimize high-quality authoritative content for voice and visual search, mobile-first design, structured data, and ongoing analytics-driven updates.
    


2.6 - [Why The Grumpy Designer Is Wary of AI’s Confidence](https://speckyboy.com/grumpy-designer-iwary-of-ais-confidence/) - Speckyboy

The Grumpy Designer warns that AI tools often present confidently phrased but potentially inaccurate answers without clear citations or safety warnings, risking user misguidance; it recommends transparency, source citations, risk warnings, and multiple solution paths.
    


2.1 - [20 Tough AWS AI Practitioner Certification Exam Questions and Answers](https://www.theserverside.com/blog/Coffee-Talk-Java-News-Stories-and-Opinions/20-Tough-AWS-AI-Practitioner-Certification-Exam-Questions-and-Answers) - The Server Side

Article offers 20 practice questions to prepare for the AWS Certified AI Practitioner exam, covering neural networks, AWS AI services (SageMaker Clarify, Bedrock, Artifact), transformer architectures, model security, prompt engineering, evaluation metrics, and responsible AI practices.
    


1.4 - [Another World (1991) game intro remade with A.I.](https://v.redd.it/9b0obopudquf1) - Reddit

An AI-assisted remake of the 1991 game Another World’s intro demonstrates video-generative AI tools like VEO and SORA producing computer-generated visuals and sound, and has prompted discussion about preserving the original style and interest in similar remakes.
    

In [21]:
# User prompt to run workflow
# user_prompt = "Run step 6, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:30:42 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting topic extraction for clustering
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/extract_topics' from Langfuse
INFO:llm:Parsed prompt 'newsagent/extract_topics': model=gpt-4.1-mini, system_len=1100, user_len=80
08:30:42 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Using model 'gpt-4.1-mini' for topic extraction
08:30:42 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Processing 63 articles for topic extraction
08:30:42 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Processing 7 chunks with concurrency 16


▶ Starting Step 6: step_06_cluster_by_topic
concurrency:  16


08:30:47 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Successfully extracted 281 total topics across articles
08:30:47 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Starting canonical topic classification for 101 topics
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/can

INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized Langfu

INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_topic': model=gpt-4.1-mini, system_len=426, user_len=179
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/canonical_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/canonical_top

...

=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        2
n_components:       42
=== Clustering Quality Metrics ===
Number of clusters: 4
Noise points: 47 (74.6%)
Average cluster size: 4.0 ± 1.0
Cluster size range: 3 - 5
=== Quality Scores ===
Silhouette Score: 0.252 (higher is better)
Calinski-Harabasz Score: 4.7 (higher is better)
Davies-Bouldin Score: 1.372 (lower is better)
HDBSCAN Validity Index: 0.055
Composite Score: 0.154 (higher is better)

[I 2025-10-13 08:31:25,492] Trial 190 finished with value: -0.15396221685047895 and parameters: {'n_components': 42, 'min_cluster_size': 3, 'min_samples': 2}. Best is trial 72 with value: -0.18549634640982715.
=== HDBSCAN Parameters ===
min_cluster_size:   3
min_samples:        2
n_components:       20
=== Clustering Quality Metrics ===
Number of clusters: 2
Noise points: 39 (61.9%)
Average cluster size: 12.0 ± 7.0
Cluster size range: 5 - 19
=== Quality Scores ===
Silhouette Score: 0.255 (higher is better)
Calinski-Harabasz Sco

/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/topic_writer' from Langfuse
INFO:llm:Parsed prompt 'newsagent/topic_writer': model=gpt-4.1, system_len=377, user_len=57


Reduced dimensions from 3072 to 22
=== Clustering Quality Metrics ===
Number of clusters: 3
Noise points: 39 (61.9%)
Average cluster size: 8.0 ± 4.2
Cluster size range: 5 - 14
=== Quality Scores ===
Silhouette Score: 0.278 (higher is better)
Calinski-Harabasz Score: 8.3 (higher is better)
Davies-Bouldin Score: 1.279 (lower is better)
HDBSCAN Validity Index: 0.081
Composite Score: 0.180 (higher is better)




08:31:26 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | 1: AI's Impact on Work and Society
08:31:26 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | The Hidden Infrastructure Trap Stalling Enterprise AI (Enterprise AI, AI Orchestration, Agents, API Integration, Infrastructure, Messaging Platforms, Governance)
The emerging new job for humans AI just created (AI Automation Engineers, Workflow Automation, Jobs And Careers, Privacy, Data Privacy, Prompt Engineering, Gen AI)
Don't panic: AI isn't here for your job (yet) (Job Automation, Jobs And Careers, AI Adoption, Labor Market, Economics, Customer Service, Gen AI)
Companies are blaming AI for layoffs &mdash; but the real reason is fear of making the wrong move, a workplace guru says (Jobs And Careers, Layoffs, AI Specialization, Labor Market, Policy And Regulation, Ethics, Generative AI)
The More Scientists Work With AI, the Less They Trust It (AI Hallucinations, Scientific Trust, Safety And Alignment, Pri

⏱️  Total execution time: 46.22s
📊 Final result:
✅ Step 6 step_06_cluster_by_topic completed successfully! Organized 63 articles into topic clusters.


In [22]:
# User prompt to run workflow
# user_prompt = "Run step 7, select section topics"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("select_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:31:28 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Free form categorization of articles
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_proposal' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_proposal': model=gpt-5-mini, system_len=638, user_len=1179


▶ Starting Step 7: step_07_select_sections


08:32:41 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Cleaning up initial categories: ['Just-In-Time App Generation', 'AI Co-Founders', 'Wayve Funding Talks', 'UK Supercomputer Program', "Scientists' Trust Decline", 'Regulating Military AI', 'Google CodeMender Security', 'AI-Run Venture Funds', 'Telecom AI Networking', 'Enterprise AI Agents', 'Other', 'AI Stocks and Market Trends', 'ChatGPT Direct Messaging', 'Big Tech AI Partnerships', 'Perplexity Pro Financials', 'Salesforce and Slack Expand AI Capabilities', 'OpenAI Ownership Complexity', 'Generative AI And Climate', "AI's Impact on Work and Society", 'Anduril Military Wearables', 'AI Workforce Transition', 'AI And Politics']
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/cat_cleanup' from Langfuse
INFO:llm:Parsed prompt 'newsagent/cat_cleanup': model=gpt-5-mini, system_len=1768, user_len=367
08:33:04 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Final categ

08:33:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Handling singleton categories
08:33:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Singleton categories (1): ['Salesforce Slack AI']
08:33:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Non-singleton categories (8): ['AI Workforce Transition', 'AI Market Trends', 'AI Societal Impact', 'Enterprise AI Agents', 'Big Tech AI Partnerships', 'AI And Politics', 'Telecom AI Networking', 'Other']
08:33:40 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Reassigning 1 singleton articles to non-singleton categories
08:33:43 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Reassigned 1 singleton articles
08:33:43 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Final category counts: [{'cat': 'AI Workforce Transition', 'count': 14}, {'cat': 'AI Market Trends', 'count': 11}, {'cat': 'AI Societal Impact', 'count': 9}, {'cat': 'Enterprise AI Age

⏱️  Total execution time: 135.29s
📊 Final result:
Categories and article counts:
cat
AI And Politics              3
AI Market Trends            11
AI Societal Impact           9
AI Workforce Transition     14
Big Tech AI Partnerships     3
Enterprise AI Agents         4
Other                        2
Telecom AI Networking        2


In [23]:
# User prompt to run workflow
# user_prompt = "Run step 8, draft sections"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("draft_sections")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:33:43 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Step 8a: Selecting stories from 48 total
08:33:43 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | 8 must-include stories with rating > 8
08:33:43 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | 40 stories with rating <= 8


▶ Starting Step 8: step_08_draft_sections


08:33:46 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Selected 40 diverse Tier 2 stories via MMR
08:33:46 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Total selected stories: 48 (target: 100)
08:33:46 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Drafting sections for 8 categories (including Other) from selected stories
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/write_section' from Langfuse
INFO:llm:Parsed prompt 'newsagent/write_section': model=gpt-5, system_len=1854, user_len=22
08:35:08 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Drafted 8 sections
08:35:08 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Reassigned 4 pruned stories to Other section
08:35:08 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Created newsletter_section_df with 48 stories across 8 categories
08:35:08 | NewsletterAgent.test_newsletter_20251013081415183040 | 

08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |     REWRITE id=10: Defines an emerging role shaping workflows; headline should be concise and active.
08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |       Old: Why companies are hiring AI automation engineers to embed, prototype, and scale everyday workflow gains
08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |       New: Companies hire AI automation engineers to prototype, embed, and scale workflow gains
08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |     DROP id=52: Source lacks authority (LADbible) and overlaps with stronger layoff/role-change pieces.
08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |     REWRITE id=16: Balances the layoffs narrative with expert context; tighten phrasing.
08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |       Old: Expert: Firms blaming AI for layoffs are driv

08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |   Telecom AI Networking: Coherence=6.5/10, Quality=6.0/10
08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |     REWRITE id=14: Headline is long and promo-like; tighten to active voice with concrete phrasing under 25 words.
08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |       Old: ZTE unveils AI Core, Wi‑Fi 7, and AI-driven optical at Milan Global Summit & User Congress 2025, rallying 500+ ICT leaders to full-stack intelligent computing.
08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |       New: ZTE unveils AI Core, Wi‑Fi 7 gear, and AI‑driven optical at Milan summit
08:37:11 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO |     MOVE id=42 to AI Market Trends: Focus is bank–telco convergence and GenAI use cases across industries, not telco operators/vendors leaning into AI. Better framed as a market trend; headline is also overl

⏱️  Total execution time: 207.93s
📊 Final result:
Drafted 36 stories across 8 sections


In [24]:
newsletter_section_df = state.newsletter_section_df.sort_values(["sort_order", "rating"],ascending=[True, False])

newsletter_section_df

,cat,section_title,id,headline,rating,prune,links,sort_order
0,Enterprise AI Agents,Enterprise AI Agents Ascend,8,Salesforce launches Agentforce 360 to deploy A...,12.010228,False,[SiliconANGLE](https://siliconangle.com/2025/1...,0
1,Enterprise AI Agents,Enterprise AI Agents Ascend,3,Slack upgrades Slackbot into AI assistant with...,10.405703,False,[The Verge](https://www.theverge.com/news/7978...,0
3,Enterprise AI Agents,Enterprise AI Agents Ascend,6,Opinion: Enterprise AI adoption stalls on inte...,8.621355,False,[Hacker Noon](https://hackernoon.com/the-hidde...,0
2,Enterprise AI Agents,Enterprise AI Agents Ascend,55,FIS integrates Glia’s AI into Digital One to a...,4.392957,False,[Retail Banker International](https://www.reta...,0
25,AI Market Trends,"Agents, Chips, and Shakeouts",32,Gartner sees agentic AI glut and looming shake...,7.496219,False,[IT Pro](https://www.itpro.com/technology/arti...,1
27,AI Market Trends,"Agents, Chips, and Shakeouts",41,Analysts back Nvidia on AI GPUs and $100B Open...,6.549803,False,[TipRanks](https://www.tipranks.com/news/nvidi...,1
35,AI Market Trends,"Agents, Chips, and Shakeouts",30,"Hoskinson Courts South Korea's $70B AI Push, P...",6.467943,False,[Tech in Asia](https://www.techinasia.com/news...,1
6,AI Market Trends,"Agents, Chips, and Shakeouts",35,Serve Robotics posts $1.8M revenue with 57 rob...,6.150542,False,[The Motley Fool](https:\/\/www.fool.com\/inve...,1
26,AI Market Trends,"Agents, Chips, and Shakeouts",38,"2026 AI outlook: integrated genAI, autonomous ...",5.987830,False,[Forbes](https://www.forbes.com/sites/bernardm...,1
24,AI Market Trends,"Agents, Chips, and Shakeouts",42,GenAI fuels bank–telco convergence: telecom da...,5.612585,False,[Retail Banker International](https://www.reta...,1


In [25]:
Markdown(state.display_newsletter().replace("$","\\\$"))


## Enterprise AI Agents Ascend

- Salesforce launches Agentforce 360 to deploy AI agents across CRM; early users report 1.8M weekly chats, 40% more proactive service. - [SiliconANGLE](https://siliconangle.com/2025/10/13/salesforce-bets-big-agents-platform-overhaul/)

- Slack upgrades Slackbot into AI assistant with search and scheduling; integrates Outlook and Google Calendar; live for 70,000 Salesforce employees, rollout by year-end. - [The Verge](https://www.theverge.com/news/797890/slack-slackbot-ai-assistant-upgrade)

- Opinion: Enterprise AI adoption stalls on integration and governance; proposes an agent-messaging layer, citing KubeMQ-Aiway, to unify APIs with observability. - [Hacker Noon](https://hackernoon.com/the-hidden-infrastructure-trap-stalling-enterprise-ai)

- FIS integrates Glia’s AI into Digital One to automate inquiries, cut wait times, and improve first-call resolution with seamless human handoff. - [Retail Banker International](https://www.retailbankerinternational.com/news/fis-glia-ai-customer-service-banking/)

...

- ClaimSorted raises $13.3M seed to triple insurance claims speed, serving 20+ insurers across U.S., U.K., EU - [Forbes](https://www.forbes.com/sites/robertolsen-1/2025/10/13/this-startup-raised-133-million-to-inject-ai-into-one-of-the-worlds-slowest-industries/)

- AI and satellites map old-forest habitats for bats, swifts; prototype March 2026, full rollout in fall - [CBC](https://www.cbc.ca/news/canada/new-brunswick/northern-hardwoods-research-institute-old-forest-critical-habitat-canadian-space-agency-9.6931814)

- AI slide tools compared: ChatGPT leads cohesion, Gemini excels at animations; none produce decks without heavy editing. - [Geeky Gadgets](https://www.geeky-gadgets.com/ai-tools-for-professional-presentation-slides/)

- Assembly coding touted to boost AI efficiency, with examples from DeepSeek and classic game optimizations. - [Wired](https://www.wired.com/story/programming-assembly-artificial-intelligence/)

AttributeError: 'NoneType' object has no attribute 'replace'

In [26]:
# User prompt to run workflow
# user_prompt = "Run step 9, finalize newsletter"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("finalize_newsletter")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


08:38:21 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Step 9: write title
08:38:21 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Compiled 36 items into markdown input
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/generate_newsletter_title' from Langfuse
INFO:llm:Parsed prompt 'newsagent/generate_newsletter_title': model=gpt-5, system_len=1082, user_len=225


▶ Starting Step 9: step_09_finalize_newsletter


08:38:58 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Generated title: Enterprise AI Agents Expand in CRM; Compute Megadeals Intensify; Shakeout Looms
08:38:58 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Step 9b: Starting critic-optimizer loop
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/critique_newsletter' from Langfuse
INFO:llm:Parsed prompt 'newsagent/critique_newsletter': model=gpt-5, system_len=2758, user_len=647
INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/improve_newsletter' from Langfuse
INFO:llm:Parsed prompt 'newsagent/improve_newsletter': model=gpt-5, system_len=1365, user_len=195
08:38:58 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Step 9b: Iteration 1/3 - Running critique
08:41:50 | NewsletterAgent.test_newsletter_20251013081415183040 | INFO | Iteration 1: Overall score = 7.6/10
08:41:50 | NewsletterAgent.test_newsletter_202510130814

⏱️  Total execution time: 471.43s
📊 Final result:
🎉 Step 9 step_09_finalize_newsletter completed successfully!
⭐ Final quality score: 8.3/10
🔄 Completed 2 critic-optimizer iteration(s)
📰 Newsletter stored in persistent state and emailed
✅ Workflow complete! All 9 steps finished successfully.


In [27]:
display(Markdown(state.final_newsletter))

# Enterprise AI Agents Expand in CRM; Compute Megadeals Intensify; Shakeout Looms

## Enterprise AI Agents Ascend

- Salesforce launches Agentforce 360 to deploy AI agents across CRM; early users report 1.8M weekly chats and 40% more proactive service - [SiliconANGLE](https://siliconangle.com/2025/10/13/salesforce-bets-big-agents-platform-overhaul/)
- Slack upgrades Slackbot into AI assistant with search, scheduling; Outlook and Google Calendar integrations; internal rollout to 70,000 employees, public by year-end - [The Verge](https://www.theverge.com/news/797890/slack-slackbot-ai-assistant-upgrade)
- FIS integrates Glia’s AI into Digital One to automate inquiries, cut wait times, and improve first-call resolution with seamless human handoff - [Retail Banker International](https://www.retailbankerinternational.com/news/fis-glia-ai-customer-service-banking/)
- Opinion: Enterprise AI adoption stalls on integration/governance; proposes agent-messaging layer to unify APIs with observability - [Hacker Noon](https://hackernoon.com/the-hidden-infrastructure-trap-stalling-enterprise-ai)

## Compute and Cloud Megadeals

- Big Tech deepens AI alliances: OpenAI–Oracle $300B compute; Meta–Google Cloud $10B; Nvidia $100B to OpenAI; financing ties raise risks - [Business Insider](https://www.businessinsider.com/big-techs-ai-love-fest-getting-messy-openai-oracle-2025-10)
- Analysts back Nvidia on AI GPUs and $100B OpenAI pact; flag Super Micro on controls and earnings misses - [TipRanks](https://www.tipranks.com/news/nvidia-vs-super-micro-smci-which-ai-hardware-stock-will-lead-the-next-rally)
- Tesla pivots to AI, robotics, and in-house chips to cut EV costs and harden its supply chain - [DigiTimes](https://www.digitimes.com/news/a20251009PD217/tesla-ai-chips-advanced-packaging.html)

## Agentic Market and Shakeout

- Gartner warns of agentic AI glut and likely shakeout; Salesforce, Microsoft, AWS lead consolidation - [IT Pro](https://www.itpro.com/technology/artificial-intelligence/the-tech-industry-is-becoming-swamped-with-agentic-ai-solutions-analysts-say-thats-a-serious-cause-for-concern)
- Apple in talks to acquire Prompt AI’s tech and staff; Seemour app retires; HomeKit may benefit - [Verdict](https://www.verdict.co.uk/news/apple-acquire-prompt-ais-talent-and-technology/)

## Workforce vs the AI Wave

- BCG: 20% of U.S. workers use AI; time savings now, broader job reshaping in 12–18 months - [Business Insider](https://www.businessinsider.com/impact-ai-job-market-change-role-2025-10)
- U.S. economy seeks evidence that AI is lifting worker productivity - [The Wall Street Journal](https://www.wsj.com/tech/ai/ai-worker-productivity-economy-77498195)
- Companies hire AI automation engineers to prototype, embed, and scale workflow gains - [Fast Company](https://www.fastcompany.com/91408440/why-every-company-needs-an-ai-automation-engineer-and-what-we-actually-do-ai-automation-engineer-hiring)
- U.S. manufacturing lags in AI adoption, risking jobs and growth without talent and infrastructure investment - [The Washington Post](https://www.washingtonpost.com/business/2025/10/13/manufacturing-artificial-intelligence/)
- Expert: Companies blaming AI for layoffs act from fear, risking long-term damage to careers and mobility - [Business Insider](https://www.businessinsider.com/firms-blaming-ai-for-layoffs-driven-by-fear-work-guru-2025-10)
- Indiscriminate AI use can hurt productivity and reputations; train employees and deploy tools strategically - [The Washington Post](https://www.washingtonpost.com/business/2025/10/13/work-advice-ai-productivity/)

## AI Hype Meets Headwinds

- Scientists’ trust in AI falls; hallucination concerns rise to 64%; fewer expect AI to surpass humans - [Futurism](https://futurism.com/artificial-intelligence/ai-research-scientists-hype)
- Hollywood pushes Sora 2 copyright guardrails as users decry restrictions and ratings fall - [Slashdot Linux](https://entertainment.slashdot.org/story/25/10/13/0456233/hollywood-demands-copyright-guardrails-from-sora-2---while-users-complain-thats-less-fun)
- EY and Microsoft launch free AI Skills Passport in India; 10-hour modules, digital badge, 10 million to skill by 2030 - [The Hindu Business Line](https://www.thehindubusinessline.com/info-tech/ey-microsoft-launch-ai-skills-passport-to-prepare-indias-youth-for-ai-driven-future/article70154856.ece)
- Boise-area cities pilot AI for communications and design, require human review amid bias and privacy worries - [Idaho Statesman](https://www.idahostatesman.com/news/local/community/boise/article312165058.html)
- Study finds AI use under 1% of daily activity; heavier users score higher on dark triad traits - [PsyPost](https://www.psypost.org/most-people-rarely-use-ai-and-dark-personality-traits-predict-who-uses-it-more/)
- Police expand AI analysis of surveillance data, raising civil rights and privacy concerns - [Slate](https://slate.com/podcasts/what-next-tbd/2025/10/how-police-are-using-artificial-intelligence)
- AI ‘resurrection’ tool distresses users, exposing technical flaws and ethical risks - [The Washington Post](https://www.washingtonpost.com/style/2025/10/13/ai-loved-ones-grandmother/)

## Power, Policy, and Perks

- U.S. visa limits squeeze Chinese AI talent as China launches K visa; Silicon Valley giants feel the pinch - [Rest of World](https://restofworld.org/2025/chinese-talent-ai-race/)
- AI reshapes U.S. campaigns: automated fundraising and targeting amid lax oversight and rising misinformation concerns - [Schneier on Security](https://www.schneier.com/blog/archives/2025/10/ai-and-the-future-of-american-politics.html)
- Book argues Silicon Valley elites pivot to Trump, defense, and AI, pursuing sovereignty “exits” - [Politico](https://www.politico.com/news/magazine/2025/10/11/elon-musk-donald-trump-silicon-valley-book-jacob-silverman-00603682)
- Google gives EMEA university students one year of free Gemini 2.5 Pro and 2TB storage - [Google Blog](https://blog.google/products/gemini/bringing-the-best-ai-to-university-students-in-europe-the-middle-east-and-africa-at-no-cost/)
- Hoskinson courts South Korea’s $70B AI push, pitching blockchain for supply chains amid regulatory uncertainty - [Tech in Asia](https://www.techinasia.com/news/ethereum-co-founder-eyes-s-korea-for-blockchain-ai-collaboration)

## Telcos Lean Into AI

- GenAI fuels bank–telco convergence: telecom data drives automated lending, conversational banking, personalized offers, and fraud detection; requires cloud and data modernization - [Retail Banker International](https://www.retailbankerinternational.com/comment/cognitive-bank-telco-partnerships/)
- ZTE unveils AI Core, Wi‑Fi 7 gear, and AI‑driven optical at Milan summit - [The Register](https://www.theregister.com/2025/10/13/zte_hosts_global_summit/)

## Other News

- Perplexity CEO uses AI search to answer investor questions live, speeding $25.6M Series A without a pitch deck - [Business Insider](https://www.businessinsider.com/perplexity-ceo-ai-not-pitch-decks-fundraising-aravind-srinivas-2025-10)
- ClaimSorted raises $13.3M seed to triple insurance claims speed, serving 20+ insurers across U.S., U.K., EU - [Forbes](https://www.forbes.com/sites/robertolsen-1/2025/10/13/this-startup-raised-133-million-to-inject-ai-into-one-of-the-worlds-slowest-industries/)
- Serve Robotics posts $1.8M revenue with 57 robots; targets 1,000+ in 2025; Nvidia exits prior stake; Uber remains backer - [The Motley Fool](https://www.fool.com/investing/2025/10/13/why-are-nvidia-and-uber-backing-this-tiny-900-mill/)
- 2026 AI outlook: integrated genAI, autonomous agents, privacy models, synthetic data, and fights over copyright and ad monetization - [Forbes](https://www.forbes.com/sites/bernardmarr/2025/10/13/10-generative-ai-trends-in-2026-that-will-transform-work-and-life/)
- AI and satellites map old-forest habitats for bats and swifts; prototype March 2026, full rollout in fall - [CBC](https://www.cbc.ca/news/canada/new-brunswick/northern-hardwoods-research-institute-old-forest-critical-habitat-canadian-space-agency-9.6931814)
- AI slide tools compared: ChatGPT leads cohesion, Gemini excels at animations; none produce decks without heavy editing - [Geeky Gadgets](https://www.geeky-gadgets.com/ai-tools-for-professional-presentation-slides/)
- Assembly coding touted to boost AI efficiency, with examples from DeepSeek and classic game optimizations - [Wired](https://www.wired.com/story/programming-assembly-artificial-intelligence/)